In [19]:
import os
import pandas as pd
import folium
import requests

# Navegar hasta la raíz del proyecto 
WORKSPACE = os.path.abspath(os.path.join(os.getcwd(), '../../'))

# Definir la ruta de la carpeta de datos
DATA_FOLDER = os.path.join(WORKSPACE, 'data')

# 1. Cargar los datasets
data_modelo_path = os.path.join(DATA_FOLDER, 'data_modelo.csv')
cne_ideam_path = os.path.join(DATA_FOLDER, 'CNE_IDEAM.xls')

# Cargar los datasets
df_modelo = pd.read_csv(data_modelo_path)
df_ideam = pd.read_excel(cne_ideam_path)

# 2. Identificar los códigos de estación únicos
df_modelo_codes = df_modelo['CodigoEstacion'].unique()

# 3. Filtrar la información en `df_ideam` relacionada con los códigos
df_ideam_filtered = df_ideam[df_ideam['CODIGO'].isin(df_modelo_codes)]

df_ideam_filtered = df_ideam_filtered.rename(columns={
    'CODIGO': 'CodigoEstacion',
    'LATITUD': 'Latitud',
    'LONGITUD': 'Longitud',
    'NOMBRE': 'NombreEstacion',
    'DEPARTAMENTO': 'Departamento',
    'MUNICIPIO': 'Municipio'
})

# 4. Crear el mapa interactivo
mapa = folium.Map(location=[df_ideam_filtered['Latitud'].mean(), df_ideam_filtered['Longitud'].mean()], zoom_start=6)

# Añadir puntos de las estaciones al mapa
for _, row in df_ideam_filtered.iterrows():
    estacion_id = row['CodigoEstacion']
    nombre = row['NombreEstacion']
    lat = row['Latitud']
    lon = row['Longitud']
    departamento = row['Departamento']
    municipio = row['Municipio']

    # Información de la estación con botón para obtener predicciones
    info = f"""
    <b>Estación:</b> {nombre}<br>
    <b>Departamento:</b> {departamento}<br>
    <b>Municipio:</b> {municipio}<br>
    <b>Codigo:</b> {estacion_id}<br>
    <button onclick="on_click('{estacion_id}')">Obtener Predicción</button>
    """

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(info, max_width=300),
        tooltip=f"Estación: {nombre} ({estacion_id})"
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save(DATA_FOLDER+'/visor.html')

# Añadir la función JavaScript al archivo HTML
with open(DATA_FOLDER+'/visor.html', 'a') as f:
    f.write("""
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script>
    function on_click(estacion_id) {
        fetch(`http://localhost:1304/prediccion/${estacion_id}`, {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            }
        })
        .then(response => {
            if (!response.ok) {
                throw new Error('Network response was not ok ' + response.statusText);
            }
            return response.json();
        })
        .then(data => {
            console.log(data);
            // Crear la gráfica de valores Reales vs Predicciones
            var fechas = Object.keys(data);
            var valoresReales = fechas.map(fecha => data[fecha].Real);
            var valoresPredicciones = fechas.map(fecha => data[fecha].Prediccion);

            var ctx = document.createElement('canvas');
            ctx.id = "myChart";
            ctx.style.width = "100%";   // Mantener la proporción de dimensiones
            ctx.style.height = "100%";  // Mantener la proporción de dimensiones
            document.body.appendChild(ctx);

            var myChart = new Chart(ctx, {
                type: 'line',
                data: {
                    labels: fechas,
                    datasets: [{
                        label: 'Real',
                        data: valoresReales,
                        borderColor: 'rgba(75, 192, 192, 1)',
                        borderWidth: 1,
                        fill: false
                    },
                    {
                        label: 'Predicción',
                        data: valoresPredicciones,
                        borderColor: 'rgba(255, 99, 132, 1)',
                        borderWidth: 1,
                        fill: false
                    }]
                },
                options: {
                    responsive: false,  // Para evitar redimensionamiento automático
                    maintainAspectRatio: true,  // Mantener la proporción de aspecto
                    scales: {
                        x: {
                            title: {
                                display: true,
                                text: 'Fecha'
                            },
                            ticks: {
                                autoSkip: true,
                                maxTicksLimit: 10
                            }
                        },
                        y: {
                            title: {
                                display: true,
                                text: 'Precipitación (mm)'
                            },
                            ticks: {
                                beginAtZero: true
                            }
                        }
                    }
                }
            });

            // Mostrar la gráfica superpuesta al mapa con botón para cerrar
            var chartContainer = document.createElement('div');
            chartContainer.style.position = "absolute";
            chartContainer.style.top = "10%";
            chartContainer.style.left = "10%";
            chartContainer.style.width = "80%";
            chartContainer.style.height = "80%";
            chartContainer.style.backgroundColor = "white";
            chartContainer.style.zIndex = "1000";
            chartContainer.style.padding = "20px";
            chartContainer.style.boxShadow = "0px 0px 10px rgba(0,0,0,0.5)";
            
            var closeButton = document.createElement('button');
            closeButton.innerHTML = "Cerrar";
            closeButton.style.position = "absolute";
            closeButton.style.top = "10px";
            closeButton.style.right = "10px";
            closeButton.onclick = function() {
                document.body.removeChild(chartContainer);
                document.body.removeChild(ctx);
            };

            chartContainer.appendChild(closeButton);
            chartContainer.appendChild(ctx);
            document.body.appendChild(chartContainer);
        })
        .catch(error => {
            console.error('Error al obtener la predicción:', error);
            alert('Error al obtener la predicción.');
        });
    }
    </script>
    """)